In [ ]:
import cv2
import numpy as np
import datetime

# Load the image
image = cv2.imread("../ComputerVision/photos/Smarties2.jpg", 1)
image = cv2.resize(image, (800, 400))
image_copy = image.copy()

roi = None
rect_start = None
circle_center = None
translation_start = None
history = []


# Mouse callback function
def mouse_callback(event, x, y, flags, param):
    global image, image_copy, roi, rect_start, circle_center, translation_start

    if event == cv2.EVENT_LBUTTONDOWN:
        rect_start = (x, y)

    elif event == cv2.EVENT_LBUTTONUP:
        cv2.rectangle(image_copy, rect_start, (x, y), (0, 255, 0), 2)
        roi = image_copy[
            min(rect_start[1], y) : max(rect_start[1], y),
            min(rect_start[0], x) : max(rect_start[0], x),
        ]

    elif event == cv2.EVENT_RBUTTONDOWN:
        circle_center = (x, y)

    elif event == cv2.EVENT_RBUTTONUP:
        cv2.circle(
            image_copy,
            circle_center,
            int(np.sqrt((circle_center[0] - x) ** 2 + (circle_center[1] - y) ** 2)),
            (0, 0, 255),
            2,
        )

    elif event == cv2.EVENT_MBUTTONDOWN:
        translation_start = (x, y)

    elif event == cv2.EVENT_MBUTTONUP:
        dx = x - translation_start[0]
        dy = y - translation_start[1]
        rows, cols = image_copy.shape[:2]
        translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
        image_copy = cv2.warpAffine(image_copy, translation_matrix, (cols, rows))


# Display current time on frame
def display_current_time(frame):
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(
        frame,
        str(current_time),
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 255),
        2,
        cv2.LINE_AA,
    )


# Reset image to its original state
def reset_image():
    global image_copy, history
    image_copy = image.copy()
    history = []


# Undo the previous operation
def undo_operation():
    global image_copy, history
    if len(history) > 0:
        image_copy = history.pop()


# Save image to disk
def save_image():
    global image_copy
    cv2.imwrite("../ComputerVision/photos/modified_image.jpg", image_copy)
    print("Image saved successfully.")


# Crop the region of interest defined by the rectangle
def crop_image():
    global roi, history, image_copy
    if roi is not None:
        history.append(image_copy)
        image_copy = roi.copy()
        roi = None


# Keyboard event callback function
def keyboard_callback(key):
    global image_copy
    
    if key == ord("g"):
        history.append(image_copy)
        gray_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
        image_copy = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)

    elif key == ord("r"):
        reset_image()

    elif key == ord("s"):
        save_image()

    elif key == ord("c"):
        crop_image()

    elif key == ord("z"):
        undo_operation()

    elif key == ord("h"):
        help_message = ["Keyboard Shortcuts:",
                        "g - Convert image to grayscale",
                        "r - Reset image to original state",
                        "s - Save the image to disk",
                        "c - Crop the region of interest",
                        "z - Undo the previous operation",
                        "q - Quit the program"]
        pos = 0
        for txt in help_message:
            cv2.putText(image_copy, txt,
                        (10, 300 + pos), cv2.FONT_HERSHEY_SIMPLEX,
                        0.4, (0, 0, 0), 1, cv2.LINE_AA
                        )
            pos+=15

def nothing(x):
    pass

# Create a named window and set the mouse callback function
cv2.namedWindow("Image")
cv2.namedWindow("Tracking")
cv2.setMouseCallback("Image", mouse_callback)

cv2.createTrackbar('Brightness', 'Tracking', 127, 127*2 , nothing)
cv2.createTrackbar('Contrast', 'Tracking', 10, 20, nothing)
cv2.createTrackbar('Saturation', 'Tracking', 127, 127*2, nothing)

# Main loop
while True:
    # Display the current time on the frame
    display_current_time(image_copy) 
    
    # Adjust Brightness, Contrast, Saturation
    hsv = cv2.cvtColor(image_copy, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    
    sat = np.array(cv2.getTrackbarPos('Saturation', 'Tracking')-127).astype(np.uint8)
    s+= sat
    s = np.clip(s,0,255)
    val = np.array(cv2.getTrackbarPos('Brightness', 'Tracking') -127).astype(np.uint8)
    v+= val
    v = np.clip(v,0,255)
    hsv = cv2.merge((h,s,v))
    
    adjusted_1 = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)  
    Contrast = cv2.getTrackbarPos('Contrast', 'Tracking')/10
    adjusted_2 = cv2.convertScaleAbs(adjusted_1, alpha=Contrast, beta=0)
 
    # Display the image
    cv2.imshow("Image", adjusted_2)

    # Wait for a key event
    key = cv2.waitKey(1)

    # Process keyboard events
    if key == ord("q"):
        break
    else:
        keyboard_callback(key)

cv2.destroyAllWindows()